In [45]:
import sys
sys.path.extend(['d:\\atom\\python'])
import data_manager as dm
import clean_data
from datetime import datetime
from statistics import mode
from collections import defaultdict
import pickle

import glob, os

plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

folder = f"d:\\atom\\exp\\{datetime.today().strftime('%Y%m%d')}"

epc = {'water': dm.epc['water'], 'oil': dm.epc['oil']}

In [46]:
features = defaultdict(lambda: {})
for file in glob.glob(os.path.join(folder, '*_h.csv')):
    print(file)
    basename = os.path.basename(file)
    matl = basename[:basename.find('_')]
    df = pd.read_csv(file).groupby('EPC').get_group(epc[matl]).drop(columns=['EPC']).sort_values(by=['CHANNEL'])
    df = clean_data.kde_peak(df)
    if len(df) != 50:
        raise Exception(f'{len(df)} channels!')
    features[matl]['h'] = [df['PHASE'].values, df['RSSI'].values]
    
for file in glob.glob(os.path.join(folder, '*_t.csv')):
    basename = os.path.basename(file)
    matl = basename[:basename.find('_')]
    df = pd.read_csv(file).groupby('EPC').get_group(epc[matl]).drop(columns=['EPC']).sort_values(by=['CHANNEL'])
    df = clean_data.kde_peak(df)
    if len(df) != 50:
        raise Exception(f'{len(df)} channels!')
    features[matl]['t'] = [df['PHASE'].values, df['RSSI'].values]

d:\atom\exp\20210515\oil_h.csv
d:\atom\exp\20210515\water_h.csv


In [37]:
# 训练分类器
d_p = []
d_r = []
y = []

for matl in features.keys():
    head = features[matl]['h']
    back = features[matl]['t']
    
#     phase_diff = head[0] - back[0]
    phase_diff = np.unwrap(head[0]) - np.unwrap(back[0])
    rssi_diff = head[1] - head[0]
    
    features[matl] = [phase_diff, rssi_diff]
    
    d_p.extend(phase_diff)
    d_r.extend(rssi_diff)
    y.extend(np.repeat([matl], len(phase_diff)))
    
    print(matl)
#     plt.scatter(phase_diff, rssi_diff)

X = np.column_stack((d_p, d_r))

# from sklearn.svm import SVC
# clf = SVC(kernel='rbf', C=1E6).fit(X, y)

# from sklearn.tree import DecisionTreeClassifier
# clf = DecisionTreeClassifier().fit(X, y)

from sklearn.naive_bayes import GaussianNB
clf = GaussianNB().fit(X, y)

# pickle.dump(clf, open(os.path.join(folder, 'model.joblib'), 'wb'))
pickle.dump(features, open(os.path.join(folder, 'features.txt'), 'wb'))

ValueError: Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.

In [34]:
matl = 'water'

epc = dm.epc[matl]
df_f = pd.read_csv(os.path.join(folder, f'{matl}_h.csv')).groupby('EPC').get_group(epc).drop(columns=['EPC'])
df_t = pd.read_csv(os.path.join(folder, f'{matl}_t.csv')).groupby('EPC').get_group(epc).drop(columns=['EPC'])

df_f = clean_data.kde_peak(df_f)
df_t = clean_data.kde_peak(df_t)

diff_p = np.unwrap(df_f['PHASE']) - np.unwrap(df_t['PHASE'])
diff_r = df_f['RSSI'] - df_t['RSSI']
ans = clf.predict(np.column_stack((diff_p, diff_r)))
print(mode(ans))
print(ans)

water
['water' 'water' 'water' 'water' 'water' 'water' 'water' 'water' 'water'
 'water' 'water' 'water' 'water' 'water' 'water' 'water' 'water' 'water'
 'water' 'water' 'water' 'water' 'water' 'water' 'water' 'water' 'water'
 'water' 'water' 'water' 'water' 'water' 'water' 'water' 'water' 'water'
 'water' 'water' 'water' 'water' 'water' 'water' 'water' 'water' 'water'
 'water' 'water' 'water' 'water' 'water']


In [44]:
 pd.read_csv(file).groupby('EPC').get_group(epc[matl])

TypeError: string indices must be integers